<a href="https://colab.research.google.com/github/alfirdevs/bayes_rule_target_detection/blob/main/Simulation4IGARSS24_k_CFAR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#############################################################################################################
#  Demo for the Bayes' Rule and its sub-optimal variants
#  -- Simulation for clutter with different sea states
#  -- Simulation for different ship sizes
#  -- Ability to control signal gain/attenuation (SNR)
#  -- Ability to choose PFA
#  -- Ability to control NESZ
#  -- k-CFAR
#  -- ROC curves
#
#     By: Khalid El-Darymli
#     Khalid.El-Darymli@drdc-rddc.gc.ca
#     21.06.2024
#     Ottawa, ON, Canada
#
#############################################################################################################

import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import gamma, rayleigh, kstwobign
import seaborn as sns
import ipywidgets as widgets
from ipywidgets import interact, VBox, HTML
from sklearn.metrics import roc_curve, auc


# Ensure plots display in the notebook
%matplotlib inline

# Simulated SAR data for different sea states with increased variability
np.random.seed(0)

def generate_clutter_data(nesz):
    nesz_linear = 10**(nesz / 10)
    return {
        'Sea State 1: Calm (rippled) - Wave Height: 0 - 0.1m': gamma.rvs(a=2, scale=1, size=1000) + nesz_linear,
        'Sea State 2: Smooth (wavelets) - Wave Height: 0.1 - 0.5m': gamma.rvs(a=2, scale=2, size=1000) + nesz_linear,
        'Sea State 3: Slight - Wave Height: 0.5 - 1.25m': gamma.rvs(a=2, scale=3, size=1000) + nesz_linear,
        'Sea State 4: Moderate - Wave Height: 1.25 - 2.5m': gamma.rvs(a=2, scale=4, size=1000) + nesz_linear,
        'Sea State 5: Rough - Wave Height: 2.5 - 4.0m': gamma.rvs(a=2, scale=5, size=1000) + nesz_linear
    }

# Define RCS values for different targets (in dBsm) and their lengths in meters
target_info = {
    'None': (None, 0),
    'Fishing Vessel (RCS=8 dBsm, Length=30 feet)': (8, 9.144),  # 30 feet
    'Smaller Coaster (RCS=20 dBsm, Length=140 feet)': (20, 42.672),  # 140 feet
    'Warship (Frigate) (RCS=40 dBsm, Length=340 feet)': (40, 103.632),  # 340 feet
    'Bulk Carrier (RCS=45 dBsm, Length=650 feet)': (45, 198.12)  # 650 feet
}

# Ground resolution of the SAR system in meters (example: 10 meters)
ground_resolution = 10

def estimate_pixels(target_length, ground_resolution):
    """
    Estimate the number of pixels based on the target size and sensor resolution.

    :param target_length: Length of the target in meters
    :param ground_resolution: Ground resolution of the SAR system in meters
    :return: Estimated number of pixels
    """
    return max(1, int(np.ceil(target_length / ground_resolution)))

def generate_target_data(rcs, snr_db, size, pixels, nesz):
    """
    Generate synthetic backscatter data for a given target RCS using Rayleigh distribution.

    :param rcs: Radar Cross Section value in dBsm
    :param snr_db: Signal to Noise Ratio in dB
    :param size: Number of samples to generate
    :param pixels: Number of pixels estimated for the target
    :param nesz: Noise Equivalent Sigma Zero in dB
    :return: Numpy array of synthetic target data
    """
    nesz_linear = 10**(nesz / 10)
    snr_linear = 10**(snr_db / 10)
    adjusted_rcs = rcs * snr_linear
    target_signal = rayleigh.rvs(loc=adjusted_rcs + nesz_linear, scale=1.5, size=size*pixels)
    clutter_noise = gamma.rvs(a=2, scale=2, size=size*pixels) + nesz_linear  # assuming clutter follows the same distribution as sea states
    return target_signal + clutter_noise

def compute_cfar_threshold(data, pfa):
    """
    Compute CFAR threshold based on K-distribution and Probability of False Alarm (PFA).

    :param data: Clutter data
    :param pfa: Probability of False Alarm
    :return: CFAR threshold
    """
    k_dist = kstwobign()
    threshold = k_dist.ppf(1 - pfa)
    scale_factor = np.mean(data) / k_dist.mean()
    return threshold * scale_factor

def compute_roc_curve(clutter_data, target_data, cfar_threshold):
    """
    Compute ROC curve data.

    :param clutter_data: Clutter data
    :param target_data: Target data
    :param cfar_threshold: CFAR threshold
    :return: False positive rate, True positive rate, and Area Under the Curve (AUC)
    """
    labels = np.concatenate([np.zeros_like(clutter_data), np.ones_like(target_data)])
    scores = np.concatenate([clutter_data, target_data])
    detections = scores >= cfar_threshold
    fpr, tpr, _ = roc_curve(labels, detections)
    roc_auc = auc(fpr, tpr)
    return fpr, tpr, roc_auc

def plot_sea_state(sea_state, pfa, target, snr_db, nesz, include_histogram, include_roc):
    """
    Plot histograms, KDE, CFAR threshold, and optionally ROC curve for given sea state and target.

    :param sea_state: Selected sea state
    :param pfa: Probability of False Alarm
    :param target: Selected target
    :param snr_db: Signal to Noise Ratio in dB
    :param nesz: Noise Equivalent Sigma Zero in dB
    :param include_histogram: Boolean to include histogram in plot
    :param include_roc: Boolean to include ROC curve in plot
    """
    clutter_sea_states = generate_clutter_data(nesz)
    plt.figure(figsize=(14, 10))

    if sea_state == 'All':
        colors = sns.color_palette("hsv", len(clutter_sea_states))
        for i, (key, data) in enumerate(clutter_sea_states.items()):
            if include_histogram:
                sns.histplot(data, bins=30, kde=True, label=f'{key} ($\\omega_B$)', color=colors[i], stat='density', alpha=0.5)
            else:
                sns.kdeplot(data, label=f'{key} ($\\omega_B$)', color=colors[i], linewidth=2)
        clutter_data = np.concatenate(list(clutter_sea_states.values()))
    else:
        clutter_data = clutter_sea_states[sea_state]
        if include_histogram:
            sns.histplot(clutter_data, bins=30, kde=True, label=f'{sea_state} ($\\omega_B$)', color='blue', stat='density', alpha=0.5)
        else:
            sns.kdeplot(clutter_data, label=f'{sea_state} ($\\omega_B$)', color='blue', linewidth=2)

    if target != 'None':
        rcs, target_length = target_info[target]
        pixels = estimate_pixels(target_length, ground_resolution)
        target_data = generate_target_data(rcs, snr_db, size=1000, pixels=pixels, nesz=nesz)
        combined_data = np.concatenate([clutter_data, target_data])
        if include_histogram:
            sns.histplot(combined_data, bins=30, kde=True, label=f'{target} ($\\omega_T$)', color='turquoise', stat='density', alpha=0.5)
        else:
            sns.kdeplot(combined_data, label=f'{target} + Clutter ($\\omega_T$)', color='turquoise', linewidth=2)

    if sea_state != 'All':
        cfar_threshold = compute_cfar_threshold(clutter_data, pfa)
        plt.axvline(cfar_threshold, color='red', linestyle='--', linewidth=3, label=f'CFAR Threshold ($\\alpha$ = {cfar_threshold:.2f})')

    plt.title('Clutter and Ship Target Distributions with KDE Envelopes and CFAR Threshold', fontsize=20)
    plt.xlabel('Backscatter Value (dB)', fontsize=18)
    plt.ylabel('Probability Density', fontsize=18)
    plt.legend(fontsize=16)
    plt.grid(True)
    plt.xticks(fontsize=14)
    plt.yticks(fontsize=14)
    plt.show()

    if include_roc and sea_state != 'All' and target != 'None':
        fpr, tpr, roc_auc = compute_roc_curve(clutter_data, target_data, cfar_threshold)
        plt.figure(figsize=(14, 10))
        plt.plot(fpr, tpr, color='blue', lw=3, label=f'ROC curve (AUC = {roc_auc:.2f})')
        plt.plot([0, 1], [0, 1], color='grey', lw=3, linestyle='--')
        plt.xlim([0.0, 1.0])
        plt.ylim([0.0, 1.05])
        plt.xlabel('False Positive Rate', fontsize=18)
        plt.ylabel('True Positive Rate', fontsize=18)
        plt.title('Receiver Operating Characteristic (ROC) Curve', fontsize=20)
        plt.legend(loc='lower right', fontsize=16)
        plt.grid(True)
        plt.xticks(fontsize=14)
        plt.yticks(fontsize=14)
        plt.show()

# Create interactive plot
initial_clutter_data = generate_clutter_data(-25)
sea_state_options = ['All'] + list(initial_clutter_data.keys())
target_options = list(target_info.keys())

# Create interactive widgets
sea_state_widget = widgets.Dropdown(options=sea_state_options, value='All', description='Sea State:')
pfa_widget = widgets.FloatLogSlider(value=1e-10, base=10, min=-20, max=0, step=0.1, description='PFA')
target_widget = widgets.Dropdown(options=target_options, value=target_options[0], description='Target:')
snr_widget = widgets.FloatSlider(value=0, min=-10, max=30, step=1, description='SNR (dB)')
nesz_widget = widgets.FloatSlider(value=-25, min=-30, max=-15, step=0.5, description='NESZ (dB)')
hist_widget = widgets.Checkbox(value=True, description='Include Histogram')
roc_widget = widgets.Checkbox(value=True, description='Include ROC Curve')

# Disable ROC and PFA widgets when "All" is selected or "None" is selected for target
def on_selection_change(change):
    if sea_state_widget.value == 'All':
        pfa_widget.disabled = True
        roc_widget.disabled = True
        roc_widget.value = False
    elif target_widget.value == 'None':
        pfa_widget.disabled = False
        roc_widget.disabled = True
        roc_widget.value = False
    else:
        pfa_widget.disabled = False
        roc_widget.disabled = False

# Initial disable for first run
pfa_widget.disabled = True
roc_widget.disabled = True
roc_widget.value = False

sea_state_widget.observe(on_selection_change, names='value')
target_widget.observe(on_selection_change, names='value')

# Create interactive output
output = widgets.interactive_output(plot_sea_state, {
    'sea_state': sea_state_widget,
    'pfa': pfa_widget,
    'target': target_widget,
    'snr_db': snr_widget,
    'nesz': nesz_widget,
    'include_histogram': hist_widget,
    'include_roc': roc_widget
})

# Display the table and the interactive plot
display(VBox([sea_state_widget, pfa_widget, target_widget, snr_widget, nesz_widget, hist_widget, roc_widget, output]))


